In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
#from fastai.tabular.core import df_shrink

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

#from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier
#from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score


In [ ]:
df = pd.read_csv(r'C:\Users\soura\Downloads\BDAT-1011 Data Analytics Project\ibm_data.csv')
df.head()

In [ ]:
#df['Target'] = df.apply(lambda _: '', axis=1)
#df.head()

In [ ]:
y = df.target
X = df.drop('target', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)
print("\nX_train:\n")
print(X_train.head())
print(X_train.shape)

print("\nX_test:\n")
print(X_test.head())
print(X_test.shape)


In [ ]:
df_train=X_train
df_test=X_test 

df_train = df_train.drop_duplicates()
df_test = df_test.drop_duplicates()




In [ ]:
df_total = df_train.append(df_test)


In [ ]:
df_train.shape



In [ ]:
df_test.shape

In [ ]:
df_total.info()


In [ ]:
print('=' * 100 + '||')
for col in df_total.columns:
    colList = list(df_total[col].value_counts().keys())
    N = 100
    colList_N = [x for index, x in enumerate(colList) if index < N]
    print(f"Column name         : {col}")
    print(f"Data Type           : {df_total[col].dtypes}")
    print(f"Count               : {len(colList)}")
    print(f"20 Unique Values    : {colList_N}")
    print(f"Percentage Frequency:")
    percent = np.round(df_total[col].value_counts() * 100 / len(df_total), 2)
    print(percent)
    print('=' * 100 + '||')

In [ ]:
print("Total of No. Of unique IP's is =",df_total.ip_addr.nunique())


In [ ]:
# IP Addres's that has total views greater than 24 in a day
ip_views = pd.DataFrame(df_total.groupby('ip_addr').VIEWS.sum().sort_values())
unique_ip_address = list(ip_views[ip_views.VIEWS > 24].index)

# Limiting the Dataset to those rows that contain one of the ip's present in unique_ip_address 
new_data = df_total[df_total.ip_addr.isin(unique_ip_address)]

# Taking intersection of ip's
#unique_ip_address = list(new_data_details.ip_addr.unique())

# These are the filterd IP's on which we have to find Infomation.
print("No. Of unique ip's {}".format(len(unique_ip_address)))

# Examples of unique ip address
unique_ip_address[:10]
#df_total = df_train.append(df_test)


In [ ]:
#df_total['page_vw_ts'] = pd.to_datetime(df_total['page_vw_ts'])
#df_total['hour'] = pd.to_datetime(df_total['page_vw_ts']).dt.hour
#df_total['minute'] = pd.to_datetime(df_total['page_vw_ts']).dt.minute

In [ ]:
#from sklearn.impute import SimpleImputer
#mode_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
#df_total[['hour','minute']] = mode_imputer.fit_transform(df_total[['hour', 'minute']])


In [ ]:
#df_total.head()


In [ ]:
df_total['user_agent'] = df_total['user_agent'].astype(str)

In [ ]:
from device_detector import SoftwareDetector


def parse_family(X):
    return SoftwareDetector(X).parse().client_name()


def parse_os(X):
    return SoftwareDetector(X).parse().os_name()


df_total["user_browser"] = df_total.user_agent.apply(parse_family)
df_total["user_os"] = df_total.user_agent.apply(parse_os)

df_total['os_browser'] = df_total["user_os"] + '_' + df_total["user_browser"]

In [ ]:
# Adding a Bounce_rate Column 
df_total["bounce_rate_%"] = ((df_total.VISIT - df_total.ENGD_VISIT)/df_total.VISIT)*100


In [ ]:
# Droping unnecessary columns
df_total = df_total.drop(labels=['wk', 'mth', 'yr', 'page_vw_ts'], axis=1)

In [ ]:
df_total.columns

In [ ]:
df_total.info()
df_total.to_csv('PowerBIData.csv')

In [ ]:
cat_col = [
    'ctry_name', 'intgrtd_mngmt_name', 'intgrtd_operating_team_name', 'city',
    'st', 'sec_lvl_domn', 'device_type', 'operating_sys', 'user_agent',
    'page_url', 'user_browser', 'user_os', 'os_browser'
]
num_col = [
    'VISIT', 'ENGD_VISIT', 'VIEWS',
    'bounce_rate_%']

tar_col = ['target']

ID = ['ip_addr']


In [ ]:
from category_encoders.ordinal import OrdinalEncoder

#Set up the OrdinalEncoder
OE = OrdinalEncoder()
#Fit and transform the encoder
df_total[cat_col] = OE.fit_transform(df_total[cat_col])
#df_total[ID] = OE.fit_transform(df_total[ID])

In [ ]:
df_train1 = df_total[~df_total.target.isnull()]
df_test = df_total[df_total.target.isnull()].drop(['target'], axis=1)

In [ ]:
df_train1.fillna('N/A')
df_test.fillna('N/A')

In [ ]:
dependent_variable = tar_col
independent_variables = num_col + cat_col

# Splitting the data into train and test
X_train1 = df_train1[independent_variables]
y_train1 = df_train1[dependent_variable]

X_test1  = df_test[ID + independent_variables]


# Random Forest

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#model = RandomForestClassifier(n_estimators=100)
#model.fit(X_train1, y_train1)

In [ ]:
#model.score(X_test1,y_train1)

In [ ]:
X_TEST = X_test1
X_TEST = X_TEST.drop_duplicates()

#X_TEST['page_vw_ts'] = pd.to_datetime(X_TEST['page_vw_ts'])
#X_TEST['hour']       = pd.to_datetime(X_TEST['page_vw_ts']).dt.hour
#X_TEST['minute']     = pd.to_datetime(X_TEST['page_vw_ts']).dt.minute
#X_TEST[['hour','minute']] = mode_imputer.transform(X_TEST[['hour', 'minute']])
X_TEST['VIEWS/VISIT'] = X_TEST['VIEWS']/X_TEST['VISIT']
X_TEST['user_agent'] = X_TEST['user_agent'].astype(str)
#X_TEST["user_browser"] = X_TEST.user_agent.apply(parse_family)
#X_TEST["user_os"] = X_TEST.user_agent.apply(parse_os)
#X_TEST['os_browser'] = X_TEST["user_os"] + '_' + X_TEST["user_browser"]

# Adding a Bounce_rate Column 
X_TEST["bounce_rate_%"] = ((X_TEST.VISIT - X_TEST.ENGD_VISIT)/X_TEST.VISIT)*100

# Droping unnecessary columns
#X_TEST = X_TEST.drop(labels=['wk', 'mth', 'yr', 'page_vw_ts'], axis=1)

# Encode the data
X_TEST[cat_col] = OE.transform(X_TEST[cat_col])

X_TEST = X_TEST[ID + independent_variables]

X_TEST.head()
X_TEST.shape


In [ ]:
def Submission_df(_test, _model, _name):
    Submission_df = pd.DataFrame()
    Submission_df['ip_addr'] = _test[['ip_addr']]
    Submission_df['target'] = _model.predict(_test.drop(['ip_addr'], axis=1))
    Submission_df.to_csv(_name + '.csv', index=False, encoding='utf-8')


from sklearn.ensemble import StackingClassifier

estimators = [('ET', ExtraTreesClassifier(random_state=2)),
              ('DT', DecisionTreeClassifier(random_state=100))]


regressor =  StackingClassifier(
    estimators=estimators,
    final_estimator=ExtraTreesClassifier(random_state=2))

regressor.fit(X_train1, y_train1)
Submission_df(X_TEST, regressor, "Final")

#Submission_df.score()


In [ ]:
regressor.score(X_TEST,tar_col)

In [ ]:
def Submission_df1(_test, _model, _name):
    Submission_df1 = pd.DataFrame()
    Submission_df1['ip_addr'] = _test[['ip_addr']]
    Submission_df1['target'] = _model.predict(_test.drop(['ip_addr'], axis=1))
    Submission_df1.to_csv(_name + '.csv', index=False, encoding='utf-8')


#from sklearn.ensemble import StackingClassifier

#estimators = [('ET', ExtraTreesClassifier(random_state=2)),
         #     ('DT', DecisionTreeClassifier(random_state=100))]




regressor = LogisticRegression()

regressor.fit(X_train1, y_train1)

Submission_df1(X_TEST,regressor,"Final_Logisitc")
